In [3]:
import pandas as pd
import numpy as np

def wraptable(thing, caption):
    # replace ascii bs
    thing = thing.replace("\\textasciicircum \\{a\\}", "\\textsuperscript{a}").replace("\\textasciicircum \\{b\\}", "\\textsuperscript{b}" )
    for substr in ['MUTAG', 'ER\\textsuperscript{a}', 'BA\\textsuperscript{a}', 'HK\\textsuperscript{a}', 'WC\\textsuperscript{a}', 'ForcedRB &']:
        thing = thing.replace(substr, f'\midrule \n{substr}', 1)
    thing = thing.replace("@@@", "")
    thing = thing.replace("Nmin", "$N_\\text{min}$")
    thing = thing.replace("Nmax", "$N_\\text{max}$")
    thing = thing.replace("gurobi", 'Gurobi')
    thing = thing.replace("LiftMP", 'OptGNN')
    replace = {'REDDIT-BINARY': '\midrule \nREDDIT-BIN\\textsuperscript{c}',
           'REDDIT-MULTI-12K': 'REDDIT-M-12K\\textsuperscript{c}',
            'REDDIT-MULTI-5K': 'REDDIT-M-5K\\textsuperscript{c}',
            'gen\\_n': "$N_\\text{min}$",
            'greedy': 'Greedy',
            'Type': 'Dataset',
            'dataset': 'Dataset',
            '+/-': '$\\pm$'} 
    for k, v in replace.items():
        thing = thing.replace(k, v)
    return thing
    #return "\\begin{table} \n" + thing + f"\n\\caption{{ {caption}}} \n" + "\\end{table}\n"


def do_bolding(latex_string, list_of_things):
    dict_of_things = {str(x): f'\\textbf{{{str(x)}}}' for x in list_of_things}
    for k, v in dict_of_things.items():
        latex_string = latex_string.replace(k, v)
    return latex_string
    
    
    

In [4]:
#df

In [5]:
fname = 'Table1_MC_std.csv'
#fname = 'Table1_VC_std.csv'
df = pd.read_csv(fname)
df = df[~df.dataset.isin(['RANDOM', 'ForcedRB'])]
dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
                              ['BA^{a}', 'ER^{a}', 'HK^{a}', 'WC^{a}', 'MUTAG^{b}', 'ENZYMES^{b}', 'PROTEINS^{b}',   'IMDB-BIN^{b}', 'COLLAB^{b}',]))

df.dataset = df.dataset.map(lambda x: dataset_conversion.get(x, x))
#df = pd.read_csv('Table1_VC.csv')

In [6]:
df

,dataset,gen_n,GAT,GCNN,GIN,GatedGCNN,LiftMP
0,ER^{a},50.0,0.992 +/- 0.005,0.944 +/- 0.021,0.940 +/- 0.015,0.994 +/- 0.005,0.998 +/- 0.002
1,ER^{a},100.0,0.988 +/- 0.004,0.943 +/- 0.019,0.944 +/- 0.010,0.988 +/- 0.004,0.996 +/- 0.002
2,ER^{a},400.0,0.989 +/- 0.002,0.951 +/- 0.006,0.959 +/- 0.003,0.982 +/- 0.002,0.993 +/- 0.001
3,MUTAG^{b},NaN,0.994 +/- 0.016,0.972 +/- 0.033,0.974 +/- 0.053,1.000 +/- 0.000,1.000 +/- 0.000
4,ENZYMES^{b},NaN,0.992 +/- 0.010,0.903 +/- 0.091,0.896 +/- 0.088,0.999 +/- 0.003,0.999 +/- 0.003
5,PROTEINS^{b},NaN,0.993 +/- 0.010,0.888 +/- 0.088,0.927 +/- 0.046,0.997 +/- 0.005,1.000 +/- 0.002
6,IMDB-BIN^{b},NaN,0.834 +/- 0.111,0.764 +/- 0.145,0.829 +/- 0.115,0.997 +/- 0.008,1.000 +/- 0.001
7,COLLAB^{b},NaN,0.990 +/- 0.011,0.814 +/- 0.160,0.899 +/- 0.131,0.922 +/- 0.090,0.999 +/- 0.002
8,REDDIT-BINARY,NaN,NaN,NaN,NaN,NaN,1.000 +/- 0.001
9,REDDIT-MULTI-12K,NaN,NaN,NaN,NaN,NaN,0.999 +/- 0.002


In [7]:
gen_n_dict = dict(zip([50, 100, 400, "", np.nan], [100, 200, 500, "", ""]))
nudf = df.round(2).fillna("")
nudf.gen_n = nudf.gen_n.apply(lambda x: int(x) if x != "" else x)
nudf['Nmax'] = nudf.gen_n.apply(lambda x: gen_n_dict[x])
nudf['Nmin'] = nudf.gen_n
nudf.dataset = nudf.apply(lambda row: f'{row.dataset} ({row.Nmin},{row.Nmax})', axis=1)
nudf.dataset = nudf.dataset.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)

nudf = nudf[['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']]

# get things to bold
list_of_things = []
for i, row in nudf.iterrows():
    possibilities = [float(x.split(" ")[0]) for x in row.values[2:] if x!=""]
    if fname == 'Table1_maxcut.csv':
        list_of_things.append(f'{max(possibilities):0.3f}')
    else:
        list_of_things.append(f'{min(possibilities):0.3f}')

main_body = (nudf.to_latex(index=False))
print(wraptable(do_bolding(main_body,list_of_things),fname))

\begin{tabular}{llllll}
\toprule
         Dataset &             GAT &            GCNN &             GIN &       GatedGCNN &          OptGNN \\
\midrule
 \midrule 
ER\textsuperscript{a} (50,100) & 0.992 $\pm$ 0.005 & 0.944 $\pm$ 0.021 & \textbf{0.940} $\pm$ 0.015 & 0.994 $\pm$ 0.005 & 0.998 $\pm$ 0.002 \\
ER\textsuperscript{a} (100,200) & 0.988 $\pm$ 0.004 & \textbf{0.943} $\pm$ 0.019 & 0.944 $\pm$ 0.010 & 0.988 $\pm$ 0.004 & 0.996 $\pm$ 0.002 \\
ER\textsuperscript{a} (400,500) & 0.989 $\pm$ 0.002 & \textbf{0.951} $\pm$ 0.006 & 0.959 $\pm$ 0.003 & 0.982 $\pm$ 0.002 & 0.993 $\pm$ 0.001 \\
       \midrule 
MUTAG\textsuperscript{b} & 0.994 $\pm$ 0.016 & \textbf{0.972} $\pm$ 0.033 & 0.974 $\pm$ 0.053 & \textbf{1.000} $\pm$ 0.000 & \textbf{1.000} $\pm$ 0.000 \\
     ENZYMES\textsuperscript{b} & 0.992 $\pm$ 0.010 & 0.903 $\pm$ 0.091 & \textbf{0.896} $\pm$ 0.088 & \textbf{0.999} $\pm$ 0.003 & \textbf{0.999} $\pm$ 0.003 \\
    PROTEINS\textsuperscript{b} & 0.993 $\pm$ 0.010 & \textbf{0.888} $\p

/tmp/ipykernel_1823815/489090871.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = (nudf.to_latex(index=False))


In [8]:
nudf.keys()

Index(['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP'], dtype='object')

# Table 2

In [9]:
fname = 'Table2_MC_std.csv'
#fname = 'Table2_VC_std.csv'

df = pd.read_csv(fname)
df = df[~df.Type.isin(['RANDOM', 'ForcedRB'])]

#dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
#                              ['BA^{b}', 'ER^{b}', 'PC^{b}', 'WC^{b}', 'MUTAG^{a}', 'ENZYMES^{a}', 'PROTEINS^{a}',   'IMDB-BINARY^{a}', 'COLLAB^{a}',]))

df.Type = df.Type.map(lambda x: dataset_conversion.get(x, x))#df = pd.read_csv('Table2_VC.csv')


df

# load in nikos baselines
if fname == 'Table2_VC.csv':
    x = pd.read_csv('kamis_lwd_baselines.csv')
    problems = {'er': 'ErdosRenyi_400.0',
            'ba': 'BarabasiAlbert_400.0',
            'hk': 'PowerlawCluster_400.0',
            'ws': 'WattsStrogatz_400.0',}

    for i, row in x.iterrows():
        problem = problems[row['problem']]
        df.loc[df['Unnamed: 0'] == problem, 'kamis'] = f'{row["k_solution_sizes_average"]:0.2f} ({row["k_times_average"]:0.2f})'
        df.loc[df['Unnamed: 0'] == problem, 'lwd'] = f'{row["l_solution_sizes_average"]:0.2f} ({row["l_times_average"]:0.2f})'
        
    

In [10]:
nudf = df.iloc[:,1:].round(2).fillna("")
#
#print(nudf.keys())



nudf.Nmin = nudf.Nmin.apply(lambda x: int(x) if x!="" else x)
nudf.Nmax = nudf.Nmax.apply(lambda x: int(x) if x!="" else x)
nudf.Type = nudf.apply(lambda row: f'{row.Type} ({row.Nmin},{row.Nmax})', axis=1)
nudf.Type = nudf.Type.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)
#nudf['dataset'] = nudf.Type
nudf = nudf[[x for x in nudf.keys() if x not in ['Nmin', 'Nmax', 'SDP proj'
]]]

# TODO: get things to bold
# get things to bold

list_of_things = []
'''
for i, row in nudf.iterrows():
    #print(row)
    possibilities = [float(x.split(" ")[0]) if isinstance(x, str) else x for x in row.values[2:]]
    if fname == 'Table1_maxcut.csv':
        list_of_things.append(f'{max(possibilities):0.2f}')
    else:
        list_of_things.append(f'{min(possibilities):0.2f}')
'''
list_of_things = []

nudf = nudf[['Type', 'LiftMP',]]#'greedy', 'gurobi_0.1', 'gurobi_1.0', 'gurobi_8.0']]


cols = nudf.columns
hrow1 = []
hrow2 = []
for x in cols:
    if x.startswith('gurobi'):
        hrow1.append("gurobi")
        hrow2.append(x.split("_")[1] + "s")
    else:
        hrow1.append(x)
        hrow2.append("@@@")

header = pd.MultiIndex.from_arrays([hrow1, hrow2])
nudf.columns = header

main_body = nudf.to_latex(index=False)

print(wraptable(do_bolding(main_body,list_of_things),fname))



\begin{tabular}{ll}
\toprule
            Dataset &          OptGNN \\
              &              \\
\midrule
 \midrule 
BA\textsuperscript{a} (50,100) & 0.998 $\pm$ 0.002 \\
BA\textsuperscript{a} (100,200) & 0.996 $\pm$ 0.003 \\
BA\textsuperscript{a} (400,500) & 0.993 $\pm$ 0.003 \\
 \midrule 
ER\textsuperscript{a} (50,100) & 0.998 $\pm$ 0.002 \\
ER\textsuperscript{a} (100,200) & 0.996 $\pm$ 0.002 \\
ER\textsuperscript{a} (400,500) & 0.993 $\pm$ 0.001 \\
 \midrule 
HK\textsuperscript{a} (50,100) & 0.998 $\pm$ 0.002 \\
HK\textsuperscript{a} (100,200) & 0.995 $\pm$ 0.003 \\
HK\textsuperscript{a} (400,500) & 0.994 $\pm$ 0.003 \\
 \midrule 
WC\textsuperscript{a} (50,100) & 0.998 $\pm$ 0.003 \\
WC\textsuperscript{a} (100,200) & 0.995 $\pm$ 0.003 \\
WC\textsuperscript{a} (400,500) & 0.989 $\pm$ 0.003 \\
       \midrule 
MUTAG\textsuperscript{b} & 1.000 $\pm$ 0.000 \\
     ENZYMES\textsuperscript{b} & 0.999 $\pm$ 0.003 \\
    PROTEINS\textsuperscript{b} & 1.000 $\pm$ 0.002 \\
    IMDB-BIN\t

/tmp/ipykernel_1823815/986849330.py:47: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = nudf.to_latex(index=False)


# instructions: 
2. add midrules